In [7]:
import pandas as pd
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
combined_data = pd.read_parquet('combined_data.parquet')
print("Columns with NaN values:")
print(combined_data.isnull().sum())
combined_data_clean = combined_data.dropna()

Columns with NaN values:
patient_id         0
event              0
T                  0
sex                0
race               0
ethnicity          0
min_travel_time    0
SDOH               0
education          0
income             0
mi                 0
chf                0
pvd                0
cevd               0
dementia           0
copd               0
rheumd             0
pud                0
mld                0
msld               0
diab               0
dia_w_c            0
hp                 0
mrend              0
srend              0
aids               0
hiv                0
mst                0
mal                0
Obesity            0
WL                 0
Alcohol            0
Drug               0
Psycho             0
Dep                0
dtype: int64


In [12]:
def run_cox_analysis(data):

    # Travel Time
    travel_covariates = ['min_travel_time']
    
    # Demographics
    demo_covariates = [col for col in data.columns if col.startswith(('sex_', 'race_', 'ethnicity_'))]
    
    # Comorbidity
    charlson_covariates = [
    'mi', 'chf', 'pvd', 'cevd', 'dementia', 'copd', 'rheumd', 'pud', 
    'mld', 'msld', 'diab', 'dia_w_c', 'hp', 'mrend', 'srend', 
    'aids', 'hiv', 'mst', 'mal', 'Obesity', 'WL', 'Alcohol', 'Drug', 'Psycho', 'Dep'
    ]
    
    # SDOH
    sdoh_covariates = [col for col in data.columns 
                      if col.startswith('education_')] + ['SDOH']
    
    covariates = travel_covariates + demo_covariates + charlson_covariates + sdoh_covariates

    cph = CoxPHFitter(penalizer=0.1)

    cox_data = data[covariates + ['T', 'event']]

    cph.fit(cox_data, duration_col='T', event_col='event')
    return cph

print("Event distribution:")
print(combined_data_clean['event'].value_counts())
print("\nTime variable statistics:")
print(combined_data_clean['T'].describe())

model = run_cox_analysis(combined_data_clean)

print("Cox Model Summary:")
model.print_summary()

Event distribution:
event
1    51
0    49
Name: count, dtype: int64

Time variable statistics:
count    100.000000
mean      15.284100
std        8.573717
min       -1.720000
25%        8.935000
50%       16.000000
75%       21.285000
max       34.810000
Name: T, dtype: float64
Cox Model Summary:


<lifelines.CoxPHFitter: fitted with 100 total observations, 49 right-censored observations>
             duration col = 'T'
                event col = 'event'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 100
number of events observed = 51
   partial log-likelihood = -175.63
         time fit was run = 2025-02-20 02:48:15 UTC

---
                 coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                        
min_travel_time -0.01      0.99      0.00           -0.01            0.00                0.99                1.00
mi              -0.35      0.70      0.70           -1.72            1.02                0.18                2.76
chf              0.12      1.12      0.67           -1.19            1.43                0.30                4.16
pvd             -0.79      0.45      0.96           -2.67            1.09                0.07                2.97
cevd            -0.03      0.97      0.73           -1.46            1.40                0.23                4.06
dementia        -0.93      0.39      0.66           -2.23            0.37                0.11                1.44
copd            -0.14      0.87      0.56           -1.24            0.96                0.29                2.61
rheumd          -0.76      0.47      0.82           -2.38            0.85                0.09                2.34
pud              0.59      1.80      0.65           -0.69            1.86                0.50                6.43
mld              0.54      1.72      0.78           -0.99            2.08                0.37                7.97
msld             0.25      1.28      0.94           -1.59            2.09                0.20                8.08
diab            -0.57      0.57      0.59           -1.72            0.58                0.18                1.78
dia_w_c          0.70      2.02      0.66           -0.59            1.99                0.55                7.32
hp              -0.57      0.57      0.58           -1.70            0.56                0.18                1.75
mrend            0.54      1.71      0.60           -0.64            1.72                0.53                5.57
srend           -1.19      0.30      0.74           -2.64            0.26                0.07                1.30
aids             1.08      2.95      1.03           -0.93            3.10                0.39               22.15
hiv              0.50      1.64      0.79           -1.05            2.04                0.35                7.72
mst              0.08      1.08      0.91           -1.70            1.86                0.18                6.44
mal              0.71      2.04      0.60           -0.46            1.88                0.63                6.54
Obesity          0.41      1.51      0.74           -1.04            1.86                0.35                6.43
WL               0.56      1.74      1.01           -1.42            2.54                0.24               12.64
Alcohol          0.27      1.31      0.76           -1.22            1.76                0.29                5.83
Drug            -0.04      0.96      1.30           -2.58            2.51                0.08               12.29
Psycho          -0.39      0.68      0.83           -2.02            1.24                0.13                3.46
Dep              1.20      3.33      0.82           -0.41            2.82                0.66               16.71
SDOH             0.02      1.02      0.07           -0.12            0.16                0.89                1.17

                 cmp to     z    p  -log2(p)
covariate                                   
min_travel_time    0.00 -1.12 0.26      1.93
mi                 0.00 -0.50 0.61      0.70
chf                0.00  0.17 0.86      0.21
pvd                0.00 -0.83 0.41      1.29
cevd   